In [1]:
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize,word_tokenize
import re
from os.path import isfile, join
from os import listdir

In [2]:
#Navigate to the root
%cd E:\Jupyter\Competing Models

E:\Jupyter\Competing Models


In [4]:

######################## Getting citation sentences ##############################

def get_citation_sentences(sentences, springer=True):
    author = "(?:[A-Z][A-Za-z'`-]+)"
    etal = "(?:et al.?)"
    additional = "(?:,? (?:(?:and |& )?" + author + "|" + etal + "))"
    year_num = "(?:19|20)[0-9][0-9]"
    page_num = "(?:, p.? [0-9]+)?"  # Always optional
    year = "(?:, *"+year_num+page_num+"| *\("+year_num+page_num+"\))"
    totcit = "(" + author + additional+"*" + year + ")"
    citation_sentences = []
    if springer == False:
        for sentence in sentences:
            if re.search(totcit, sentence):
                #print(re.findall(totcit, sentence))
                citation_sentences.append(sentence)
    else:
        matcher = '\[[0-9]+\]'
        for sentence in sentences:
            if re.search(matcher, sentence):
                if ("dataset" or "datasets" or "database" or "databases" or "organization" or "organizations" or "university" or "universities")not in sentence.lower():
                    citation_sentences.append(sentence)
    return totcit, citation_sentences

In [6]:
parentdir = '.\data'
textdir = "Text"

inputfolder = join(parentdir, textdir)
fileslist = [f for f in listdir(inputfolder) if isfile(
    join(inputfolder, f))]
fileslist.sort()
citsentlist, classifiedsent = [], []
for filename in fileslist:
    text = ""
    filepath = join(parentdir, "Text")+"/"+filename
    file = open(filepath, encoding='utf-8')
    raw_text = file.read()
    raw_text = raw_text.replace('-\n', '')
    raw_text = raw_text.replace('\n', ' ')
    text = text+raw_text
    
    sentences = []
    for s in sent_tokenize(text):
        sentences.append(s)
    # If there is 'et al.' at the end of the sentence, then join it with the
    # with the next sentence
    for i, s in enumerate(sentences):
        if s[-6:] == 'et al.' and i+1 < len(sentences):
            sentences[i] = s + " "+sentences[i+1]

    totcit, citation_sentences = get_citation_sentences(
        sentences, springer=False)
    citation_sentences_folder = parentdir+"\\"+"citation_sentences"
    with open(citation_sentences_folder+"\\"+filename,'w', encoding = "utf-8") as f:
        for sentence in citation_sentences:
            if sentence[0] == '(' or sentence[0] == '[' or len(word_tokenize(sentence))>300:
                continue
            f.write(sentence+"\n")

In [18]:
# Demo for loading the citation sentences from the text files
# citation_sentences_folder = parentdir+"\\"+"citation_sentences"
# citelist = []
# with open(citation_sentences_folder+"\\"+"Cubeflow.txt",'r', encoding = 'utf-8') as f:
#     for line in f:
#         citelist.append(line.strip("\n"))